In [1]:
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.models import Model
from keras.optimizers import SGD
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
df=pd.read_csv(r'data/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv')
df['image'] = df['image']+'.jpg'

classes = ['MEL','NV','BCC','AKIEC','BKL','DF','VASC']
groupwise = []
for class_ in classes:
    groupwise.append(df[df[class_]==1.0])
for name,group in zip(classes,groupwise):
    print(name,':',len(group))
    
np.random.seed(50)
train = None
test = None
validate = None
for g in groupwise:
    tr,te,va = np.split(g, [int(.95*len(g)), int(.96*len(g))])
    #print(tr)
    if train is not None:
        train =train.append(tr)
    else:
        train = tr
    if test is not None:
        test = test.append(te)
    else:
        test = te
    if validate is not None:
        validate = validate.append(va)
    else:
        validate = va

train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
validate = validate.sample(frac=1).reset_index(drop=True)
        
print('train count:',len(train))
print('train:\n',train.head())
print('test count:',len(test))
print('test:\n',test.head())
print('validate count',len(validate))
print('validate:\n',validate.head())

MEL : 1113
NV : 6705
BCC : 514
AKIEC : 327
BKL : 1099
DF : 115
VASC : 142
train count: 9511
train:
               image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0031583.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0025666.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0027174.jpg  1.0  0.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0029955.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0026651.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
test count: 100
test:
               image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0033861.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0033826.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0033822.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0033825.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0033839.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
validate count 404
validate:
               image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0034090.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0034075.jpg  0.0  1.0  0.0 

**Clearly the dataset is skewed this will be handled by adjusting the weights**

In [3]:
group_to_duplicate = []
for class_ in classes:
    group_to_duplicate.append(train[train[class_]==1.0])
for g in group_to_duplicate:
    #print(g)
    for i in range(len(g),4500,len(g)):
        train = train.append([g],ignore_index=True)
for class_ in classes:
    print(len(train[train[class_]==1.0]))

5285
6369
4880
4650
5220
4578
4556


In [4]:
IMG_HEIGHT,IMG_WIDTH = 224,224
datagen=ImageDataGenerator(rescale=1./255,
                           shear_range=0.2,zoom_range=0.2,
                           rotation_range=180,width_shift_range=0.1,
                           height_shift_range=0.1)

validategen= ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=train, directory="data/ISIC2018_Task3_Training_Input", 
                                            x_col="image", y_col=['MEL','NV','BCC','AKIEC','BKL','DF','VASC'], 
                                            class_mode="raw", 
                                            target_size=(IMG_HEIGHT,IMG_WIDTH), batch_size=64)
valid_generator=validategen.flow_from_dataframe(dataframe=validate, directory="data/ISIC2018_Task3_Training_Input", 
                                            x_col="image", y_col=['MEL','NV','BCC','AKIEC','BKL','DF','VASC'], 
                                            class_mode="raw", 
                                            target_size=(IMG_HEIGHT,IMG_WIDTH), batch_size=64)

Found 35538 validated image filenames.
Found 404 validated image filenames.


In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs=4)

Epoch 1/4
556/556 [==============================] - 614s 1s/step - loss: 1.5029 - accuracy: 0.4620 - val_loss: 1.2973 - val_accuracy: 0.6188
Epoch 2/4
556/556 [==============================] - 608s 1s/step - loss: 1.1668 - accuracy: 0.5664 - val_loss: 1.3911 - val_accuracy: 0.5668
Epoch 3/4
556/556 [==============================] - 607s 1s/step - loss: 1.0616 - accuracy: 0.6034 - val_loss: 1.5716 - val_accuracy: 0.6535
Epoch 4/4
556/556 [==============================] - 611s 1s/step - loss: 0.9879 - accuracy: 0.6282 - val_loss: 1.4616 - val_accuracy: 0.5792


In [6]:
model.save('inception_base.h5')

In [7]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [8]:
# train the model on the new data for a few epochs
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20)

Epoch 1/20
555/555 [==============================] - 622s 1s/step - loss: 0.8417 - accuracy: 0.6824 - val_loss: 1.6642 - val_accuracy: 0.5417
Epoch 2/20
555/555 [==============================] - 613s 1s/step - loss: 0.7777 - accuracy: 0.7109 - val_loss: 1.5716 - val_accuracy: 0.5588
Epoch 3/20
555/555 [==============================] - 609s 1s/step - loss: 0.7450 - accuracy: 0.7209 - val_loss: 1.2321 - val_accuracy: 0.5647
Epoch 4/20
555/555 [==============================] - 607s 1s/step - loss: 0.7069 - accuracy: 0.7364 - val_loss: 1.3383 - val_accuracy: 0.5559
Epoch 5/20
555/555 [==============================] - 610s 1s/step - loss: 0.6793 - accuracy: 0.7476 - val_loss: 1.4336 - val_accuracy: 0.5882
Epoch 6/20
555/555 [==============================] - 615s 1s/step - loss: 0.6562 - accuracy: 0.7548 - val_loss: 1.5353 - val_accuracy: 0.5500
Epoch 7/20
555/555 [==============================] - 610s 1s/step - loss: 0.6384 - accuracy: 0.7584 - val_loss: 1.0511 - val_accuracy: 0.5794

In [9]:
model.save('inception.h5')

In [ ]:
from keras.models import load_model
model=load_model('resnet152.h5')

# train the model on the new data for a few epochs
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50)

In [ ]:
model.save('resnet152_2.h5')

In [ ]:
# train the model on the new data for a few epochs
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    class_weight=weights)
model.save('resnet152_3.h5')

In [ ]:
nb_samples = len(valid_generator.filenames)

predictions = model.predict_generator(valid_generator,steps = nb_samples)
y_pred = np.rint(predictions)

In [ ]:
y_true = valid_generator.classes
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predict_this=[]
actual=[]
x,y = next(valid_generator)
for i in range(len(x)):
    #plt.imshow(x[i])
    #plt.show()
#     print('label:',y[i])
    actual.append(y[i])
    predict_this.append(x[i])
    #print('----------------------------------------------------------------------------------------')
np.set_printoptions(suppress=True)    
pred = model.predict(np.array(predict_this))

#pred = tf.cast(pred, tf.float32)
# print(pred)
# print(type(pred))
pred = np.argmax(pred, axis=1)
actual = np.argmax(actual, axis=1)
# print(pred)
# print(actual)
# if actual.all()==pred.all():
#     print('WIN!!!!!!!!!!!!!!!')
cr = classification_report(actual, pred)
cm = confusion_matrix(actual,pred)
print(cr)
print(cm)

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.models import Model
from keras.optimizers import SGD

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False 

#     model.add(Flatten())
#     model.add(Dense(4096, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(4096, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(1000, activation='softmax'))
# note we exclude the above final dense layers, and add the dense layers below, so we could retrain it ourselves

x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
            loss='categorical_crossentropy', metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                   class_weight=weights)

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (224, 224, 3)
num_classes = 7
from keras.layers import MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile the model
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                    class_weight=weights)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predict_this=[]
actual=[]
x,y = next(valid_generator)
for i in range(len(x)):
    #plt.imshow(x[i])
    #plt.show()
#     print('label:',y[i])
    actual.append(y[i])
    predict_this.append(x[i])
    #print('----------------------------------------------------------------------------------------')
np.set_printoptions(suppress=True)    
pred = model.predict(np.array(predict_this))

#pred = tf.cast(pred, tf.float32)
# print(pred)
# print(type(pred))
pred = np.argmax(pred, axis=1)
actual = np.argmax(actual, axis=1)
# print(pred)
# print(actual)
# if actual.all()==pred.all():
#     print('WIN!!!!!!!!!!!!!!!')
cr = classification_report(actual, pred)
cm = confusion_matrix(actual,pred)
print(cr)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predict_this=[]
actual=[]
x,y = next(valid_generator)
for i in range(len(x)):
    #plt.imshow(x[i])
    #plt.show()
#     print('label:',y[i])
    actual.append(y[i])
    predict_this.append(x[i])
    #print('----------------------------------------------------------------------------------------')
np.set_printoptions(suppress=True)    
pred = model.predict(np.array(predict_this))

#pred = tf.cast(pred, tf.float32)
# print(pred)
# print(type(pred))
pred = np.argmax(pred, axis=1)
actual = np.argmax(actual, axis=1)
# print(pred)
# print(actual)
# if actual.all()==pred.all():
#     print('WIN!!!!!!!!!!!!!!!')
cr = classification_report(actual, pred)
cm = confusion_matrix(actual,pred)
print(cr)
print(cm)

In [ ]:
print('loss, accuracy',model.evaluate(valid_generator))